## This file contains the code for skill gap analysis and transition recommendation based on the dataset curated and processed by Nesta.

### 1. Solr search to find best matching occupation

One of the shortcoming of Nesta project is that the user occupation has to exactly match to the occupations given in the ESCO and ONET dataset called "ESCO_occupation_profiles". We have overcome this by performing a Solr search to get the occupations which are closest possible match to the one user has searched for. For example, there is no occupation called "Software Engineer" in the ESCO dataset so we performed a Solr search to find the closest possible match to said occupation and the top result was "Software Analyst".  

How to install apache Solr: https://factorpad.com/tech/solr/tutorial/install-solr.html

Overview of Solr: https://factorpad.com/tech/solr/tutorial/solr-basics.html


1. Creating a Solr core:

To create a Solr core, use command "bin/solr create_core -c test_core" where "test_core" is the core name where the dataset to be searched in would be posted.

2. Making necessary changes 

As per tutorial provided by Apache Solr two important changes were made to the schema. One is to make sure that the field where the text whould be searched in is interpret as String and another is to make sure that the search is cathing all fields when the text is searched. Detailed steps are here: https://solr.apache.org/guide/7_0/solr-tutorial.html#exercise-2

3. Posting the datafile into the core

To post the data into the core file first a directory called "test_nesta" was created to store the dataset used called "ESCO_occupation_profiles.csv" (renamed to "ESCO_occ.csv") and then this file was posted into the core created in previous step by using command: "bin/post -c test_core bin/test_nesta/ESCO_occ.csv"

4. Query the text

On Solr UI "software engineer" was searched by typing in the "q" field and results were shown.




In [1]:
#All imports

from urllib.request import urlopen
import pandas as pd
import random

import warnings
warnings.filterwarnings('ignore')

Following is a dataset containing occupations to be used for testing occupation search. The dataset is taken from Kaggle.com

In [2]:
#dummy user data from Kaggle.com: https://www.kaggle.com/andrewmvd/occupation-salary-and-likelihood-of-automation?select=automation_data_by_state.csv
all_occupations = pd.read_csv("../../../data/occupation_automation_data_by_state.csv")
all_occupations['Occupation']

0                               Chief Executives
1                General and Operations Managers
2            Advertising and Promotions Managers
3                             Marketing Managers
4                                 Sales Managers
                         ...                    
697      Pump Operators; Except Wellhead Pumpers
698                             Wellhead Pumpers
699    Refuse and Recyclable Material Collectors
700                   Mine Shuttle Car Operators
701            Tank Car; Truck; and Ship Loaders
Name: Occupation, Length: 702, dtype: object

In [3]:
user_occupation = all_occupations['Occupation'][(random.randint(0,701))] #choosing a random row from all occupations data

#making occupation lower case
query = user_occupation.lower()
query = query.replace(";"," ") #replacing ; with space
query = query.replace(" ","%20") #replacing space with % sign as solr would not take space in the input

print(user_occupation)
query # printing the final query

First-Line Supervisors of Housekeeping and Janitorial Workers


'first-line%20supervisors%20of%20housekeeping%20and%20janitorial%20workers'

#### 1.1 Set up Solr Connection

For passing a qury to Solr we have to make sure first that there is no space in the query. Solr interprets "%" as space hence when searching for a query, say, "Software engineer" we convert it into "software%20engineer".


Searching for occupation example "Software Engineer"

In [4]:
user_occupation = "Pilot"


query = user_occupation.strip() #deleting any leading and trailing spaces
query = query.lower() #making occupation lower case
query = query.replace(";"," ") #replacing ; with space
query = query.replace(" ","%20") #replacing space with % sign as solr would not take space in the input

print(user_occupation)
query # printing the final query

Pilot


'pilot'

In [5]:
# search for a query
connection = urlopen('http://localhost:8983/solr/test_nesta/query?q={}&&wt=python&&fl=id,preferred_label,score'.format(query))
response = eval(connection.read())


In [6]:
response

{'responseHeader': {'status': 0,
  'QTime': 11,
  'params': {'q': 'pilot', 'fl': 'id,preferred_label,score', 'wt': 'python'}},
 'response': {'numFound': 9,
  'start': 0,
  'maxScore': 2.9587922,
  'numFoundExact': True,
  'docs': [{'id': '2385',
    'preferred_label': ['air force pilot'],
    'score': 2.9587922},
   {'id': '389', 'preferred_label': ['maritime pilot'], 'score': 2.5809317},
   {'id': '838', 'preferred_label': ['helicopter pilot'], 'score': 2.5809317},
   {'id': '981',
    'preferred_label': ['airline transport pilot'],
    'score': 2.5809317},
   {'id': '982', 'preferred_label': ['commercial pilot'], 'score': 2.5809317},
   {'id': '1537', 'preferred_label': ['co-pilot'], 'score': 2.5809317},
   {'id': '1655', 'preferred_label': ['private pilot'], 'score': 2.5809317},
   {'id': '1843', 'preferred_label': ['aircraft pilot'], 'score': 2.5403786},
   {'id': '2310',
    'preferred_label': ['ship pilot dispatcher'],
    'score': 2.5403786}]}}

Code snippet below generates the results of an example search for 'Software Engineer'. We can notice that occupation "software analyst" has the best matching score so we would go ahead for further analysis with the occupation 'software analyst'.

In [7]:
results = (response['response']['docs'])

for items in results:
    print(items)

{'id': '2385', 'preferred_label': ['air force pilot'], 'score': 2.9587922}
{'id': '389', 'preferred_label': ['maritime pilot'], 'score': 2.5809317}
{'id': '838', 'preferred_label': ['helicopter pilot'], 'score': 2.5809317}
{'id': '981', 'preferred_label': ['airline transport pilot'], 'score': 2.5809317}
{'id': '982', 'preferred_label': ['commercial pilot'], 'score': 2.5809317}
{'id': '1537', 'preferred_label': ['co-pilot'], 'score': 2.5809317}
{'id': '1655', 'preferred_label': ['private pilot'], 'score': 2.5809317}
{'id': '1843', 'preferred_label': ['aircraft pilot'], 'score': 2.5403786}
{'id': '2310', 'preferred_label': ['ship pilot dispatcher'], 'score': 2.5403786}


In [8]:
response['response']['docs'][4]['preferred_label'][0] # getting the first occupation from the dictionary

'commercial pilot'

### 2. Career transition 

In the codebase provided by Nesta, the file called "notebook_preamble_Transitions.ipy" is the first file that is required to run as it contains all the necessary imports from the package mapping_career_causeway. 

In [9]:
# running the necessary file 

%run ../../notebook_preamble_Transitions.ipy


In [10]:
# Occupational profiles
#data.occ.info()

#### 2.1: Code snippet below is generating the description and alternate names for occupation selected above.

In the code below object "data" is an instance of class "Data", which is declared in the "load_data_utils.py" utility file provided in the package. "data" is declared in the "notebook_preamble_Transitions.ipy" file as below:

data= load_data.Data( ) where load_data is refering to "load_data_utils.py" file 





In [11]:
# Check transitions for an occupation

occupation_label = results[4]['preferred_label'][0]


# Get the job ID number
job_i = data.occ_title_to_id(occupation_label) #Finds occupation's ID based on a string input

# Description 
data.describe_occupation(job_i) #Show occupation's description and alternative labels 

occupation ID 982: commercial pilot

---
Description:

Commercial pilots navigate flight of fixed-wing and multi-engine aircrafts for the transport of passengers and cargo.

---
Alternative occupation labels:

pilot of commercial aircraft
pilot of commercial aeroplanes
airline pilot
captain


#### 2.2: Generating the possible desirable transitions from the given occupation. "Desirable" is defined as the jobs that have better earning possibilities.

In [12]:
#@markdown Select the type of transition (strictly safe and desirable is the default)
transition_type = "desirable" #@param ["viable", "desirable", "safe_desirable", "strictly_safe_desirable"]

# Define column to use, to filter the transitions
transition_type_col = "is_" + transition_type

# Generate transitions: This code generates a table with all possible transitions and later we are filtering the column with desirable transitions 
trans = trans_utils.get_transitions(
    origin_ids=[job_i],
    less_information=True)

# Select transitions according to the specified transition_type
# (NB: Need to reset the index)
selected_transitions = trans[trans[transition_type_col]].reset_index() #this generates a table of all desirable transitions

print(f'{len(selected_transitions)} {" ".join(transition_type.split("_"))} transitions from {occupation_label}.')


# printing only destination labels from the transition table
print('\n----\n')
print('Transition destinations:\n')
for destination in selected_transitions.destination_label.to_list():
  print(destination)

26 desirable transitions from commercial pilot.

----

Transition destinations:

aircraft pilot
air traffic controller
aviation surveillance and code coordination manager
aviation safety officer
airside safety manager
airport maintenance technician
air traffic manager
aviation data communications manager
airspace manager
aviation inspector
flight operations officer
airport director
air traffic safety technician
airport environment officer
ground lighting officer
pipeline compliance coordinator
aircraft maintenance coordinator
commissioning engineer
construction safety manager
astronaut
aircraft engine specialist
pipeline environmental project manager
railway electronic technician
commissioning technician
radiation protection officer
police officer


In [13]:
selected_transitions

,index,origin_id,origin_label,destination_id,destination_label,similarity,is_viable,is_desirable,is_safe_desirable,is_strictly_safe_desirable
0,0,982,commercial pilot,1843,aircraft pilot,0.928970,True,True,True,False
1,2,982,commercial pilot,728,air traffic controller,0.523488,True,True,True,False
2,4,982,commercial pilot,1687,aviation surveillance and code coordination ma...,0.464536,True,True,True,True
3,5,982,commercial pilot,2377,aviation safety officer,0.457789,True,True,True,False
4,6,982,commercial pilot,816,airside safety manager,0.446604,True,True,True,False
5,7,982,commercial pilot,1562,airport maintenance technician,0.432825,True,True,True,True
6,9,982,commercial pilot,163,air traffic manager,0.408983,True,True,True,False
7,10,982,commercial pilot,1064,aviation data communications manager,0.402656,True,True,True,False
8,11,982,commercial pilot,2446,airspace manager,0.400641,True,True,True,False
9,12,982,commercial pilot,2076,aviation inspector,0.392292,True,True,True,False


##### Now, there are three ways to move forward:
1. Generate skill gap table and select the skills that have score below a threshold, say, 0.5 and then performing upskilling.
2. Let the user select an occupation from the list of previously generated transitions and then generate the skill difference.
3. Combine the above two ways by giving the user an option to choose from whether they would like the algorithm to generate skill gap analysis or choose a destination occupation themselves from the list of previously generated occupations (step 2.2).

In [14]:
#trans[trans[transition_type_col]]

### 3: Skill gap analysis by automated upskilling

#### 3.1: Generating skill gap table

In [15]:
# Set up the skills gap analysis class
gaps = trans_utils.SkillsGaps(selected_transitions) # "SkillGaps" is a class in the utility file "transitions_utils". This line is sending the required table of transitions
gaps.setup(
    transition_indices=selected_transitions.index.to_list(), # which transitions we wish to analyse (corresponding to the rows of the dataframe)
    skills_type='destination') # which skills matching scores to analyse (either 'destination' or 'origin')


Time elapsed: 1.85 sec ( 0.071 per transition)


In [16]:
# Check the most prevalent skills items and their average matching scores across the transitions
# (Here, we check skills in the 90th percentile of prevalence)
#line 771 of trans_util file
skill_gap_df=gaps.prevalent_skills_gaps(top_x=95, # Either the top prevalence percentile to show, or the top X skills to show (if percentile is False)
                           percentile=True
                          ).round(2)

In [17]:
skill_gap_df

,id,preferred_label,level_1,level_2,level_3,counts,prevalence,score,stdev
35,1571,perform risk analysis,S2,S2.7,S2.7.5,6,0.23,1.00,0.00
102,4383,work in an aviation team,S1,S1.8,S1.8.1,8,0.31,1.00,0.00
185,7753,write work-related reports,S1,S1.13,S1.13.3,8,0.31,1.00,0.00
190,7922,identify airport safety hazards,S2,S2.8,S2.8.4,7,0.27,1.00,0.00
209,8895,apply airport standards and regulations,S3,S3.3,S3.3.3,8,0.31,1.00,0.00
46,2037,aircraft flight control systems,K10,K10.4,K10.4.0,4,0.15,1.00,0.00
77,3475,use different communication channels,S1,S1.2,S1.2.1,10,0.38,1.00,0.00
219,9432,air transport law,K4,K4.2,K4.2.0,5,0.19,1.00,0.00
220,9473,air traffic control operations,K10,K10.4,K10.4.0,4,0.15,1.00,0.00
266,11250,common aviation safety regulations,K4,K4.2,K4.2.0,10,0.38,1.00,0.00


In [18]:
#len(skill_gap_df[skill_gap_df['score'] < 0.5]["preferred_label"].to_list())

#### 3.2: Selecting set of skills that have value of score less than 0.5 in the skill gap table and then simulating upskilling 


In [19]:
#@markdown ## Enter the occupation and the skill to upskill
#@markdown After entering the text, run the cell by pressing the 'play' button. </br>
#@markdown Note that `occupation_label` has to exactly match an [ESCO occupation](https://ec.europa.eu/esco/portal/occupation), and `skill_to_upskill` has to match an [ESCO skill](https://ec.europa.eu/esco/portal/skill).
origin_label = job_i #@param {type:"string"}
#skill_to_upskill = ["perform scientific research","develop automated migration methods","provide cost benefit analysis reports","manage database","web programming"] #@param {type:"string"}

skill_to_upskill = skill_gap_df[skill_gap_df['score'] < 0.5]["preferred_label"].to_list()
# Find origin occupation's ID
job_i = data.occ_title_to_id(origin_label)

## Try upskilling the chosen occupation with the selected skill

# NB: Testing many destination_ids will take a minute or so!
# Restrict the range of destination occupations for faster calculations,
# by only checking destination occupations that... 
# (1) ...are lower risk
destination_ids = data.occ_report[data.occ_report.risk_category!='High risk'].id.to_list()
# (2) ...are earning at least 75% of the origin occupation's annual earnings
destination_ids = set(destination_ids)
destination_ids = destination_ids.intersection(set(data.occ_report[data.occ_report.annual_earnings > 0.75*data.occ.loc[job_i].annual_earnings].id.to_list()))
# (3) ...have comparable expected education and experience (as measured by job zones)
destination_ids = destination_ids.intersection(set(data.occ_report[np.abs(data.occ_report.job_zone-data.occ.loc[job_i].job_zone) <= 1].id.to_list()))
# (4) ...are closer to the viability threshold;
destination_ids = destination_ids.intersection(set(np.where(sim.W_combined[job_i, :] > 0.2)[0]))
destination_ids = list(destination_ids)

#creating a list of skills
#skills = ["develop automated migration methods","information structure"]
if type(skill_to_upskill)== list:

  skill_ids = []
  for skill in skill_to_upskill:
    skill_ids.append(data.skill_title_to_id(skill))
else: 
  skill_ids = [data.skill_title_to_id(skill_to_upskill)]
upskilling = trans_utils.Upskilling(
    origin_ids=[job_i], # List of occupations we are testing 
    new_skillsets=[skill_ids], # List of skills or skillsets to add (here: only 1)
    destination_ids=destination_ids # List of admissible destination occupations
)

upskilling.effectiveness(safe_definition='strict');

# List of the new transition destinations
new_transitions = upskilling.upskilling_effects[0]['transition_table']
df = new_transitions[new_transitions.is_viable][['destination_label']].reset_index(drop=True)

print(f'{len(df)} new transitions found for "{occupation_label}" after adding the skill "{skill_to_upskill}"')
df

6 new transitions found for "commercial pilot" after adding the skill "['airport safety regulations', 'comply with legal regulations', 'apply technical communication skills', 'electronics']"


,destination_label
0,aircraft maintenance engineer
1,railway station manager
2,aircraft maintenance technician
3,criminal investigator
4,polygraph examiner
5,avionics technician


Note: At this point, we can also provide an option to let the user choose which occupation they want to transition to and then simulate upskilling by generating the skill difference similar to below.

### 4: Simulating upskilling by checking the skill difference between origin occupation and occupation chosen by user


#### 4.1: generating skill difference between source and destination occupations

In [20]:
# Check transitions for hotel concierge
#occupation_label = 'commercial pilot'
#occupation_label = 'ICT system analyst'


# Get the job ID number
#job_i = data.occ_title_to_id(occupation_label)

# Description 
skills_source = data.occupation_skills(job_i, skill_importance="Essential")

In [21]:
#set(skills_source["preferred_label"])

In [22]:
#taking user input for destination occupation. Should be a dropdown of available transitions.

user_choice = input("What is your destination occupation of choice: ")
user_choice = user_choice.strip()
user_choice=user_choice.lower()
user_choice

What is your destination occupation of choice: aircraft maintenance engineer


'aircraft maintenance engineer'

In [23]:
# Check transitions for data scientist
dest_occupation_label = user_choice

# Get the job ID number
dest_job_i = data.occ_title_to_id(dest_occupation_label)

# job skills 
skills_dest = data.occupation_skills(dest_job_i, skill_importance="Essential")

In [24]:
skills_dest["preferred_label"]

0                    manage health and safety standards
1                                             mechanics
2                                      airport planning
3                                    technical drawings
4                                 use testing equipment
5                                 engineering processes
6                  manage airport development resources
7                                mechanical engineering
8                                engineering principles
9      ensure compliance with airport security measures
10                                     electromechanics
11                                  disassemble engines
12                     wear appropriate protective gear
13                                          electronics
14                                          electricity
15                                       repair engines
16                          use technical documentation
17                                    engine com

In [25]:
#len(skills_dest["preferred_label"])


In [26]:
#final skills that are lacking in order to transition to new occupation

skill_diff = set(skills_dest["preferred_label"]).difference(set(skills_source["preferred_label"]))
skill_diff 

{'aircraft mechanics',
 'apply technical communication skills',
 'diagnose defective engines',
 'disassemble engines',
 'electrical engineering',
 'electrical wiring plans',
 'electricity',
 'electromechanics',
 'electronics',
 'engine components',
 'engineering principles',
 'engineering processes',
 'ensure compliance with airport security measures',
 'evaluate engine performance',
 'follow manufacturer guidelines in use of airport equipment',
 'inspect aircraft for airworthiness',
 'install electrical and electronic equipment',
 'manage airport development resources',
 'manage health and safety standards',
 'mechanical engineering',
 'mechanics',
 'read engineering drawings',
 'read standard blueprints',
 'repair engines',
 'technical drawings',
 'use power tools',
 'use technical documentation',
 'use testing equipment',
 'wear appropriate protective gear'}

In [27]:
#len(skill_diff)

#### 4.2: Clustering the generated skill difference

Since in some cases the list of skill difference can be too large, it would be better to group the similar skills in clusters for better user experience.

In [28]:
import numpy as np
from sklearn.cluster import AffinityPropagation
import Levenshtein
    
words = list(skill_diff) 
words = np.asarray(words) #So that indexing with a list will work
lev_similarity = -1*np.array([[Levenshtein.distance(w1,w2) for w1 in words] for w2 in words])

affprop = AffinityPropagation(affinity="precomputed", damping=0.5)
affprop.fit(lev_similarity) # clustering the similar words
for cluster_id in np.unique(affprop.labels_):
    exemplar = words[affprop.cluster_centers_indices_[cluster_id]]
    cluster = np.unique(words[np.nonzero(affprop.labels_==cluster_id)])
    cluster_str = ", ".join(cluster)
    print(" - *%s:* %s" % (exemplar, cluster_str))

 - *engineering processes:* engineering principles, engineering processes, evaluate engine performance, read engineering drawings
 - *manage airport development resources:* manage airport development resources
 - *wear appropriate protective gear:* wear appropriate protective gear
 - *inspect aircraft for airworthiness:* inspect aircraft for airworthiness
 - *electrical engineering:* electrical engineering, electrical wiring plans, mechanical engineering, use technical documentation
 - *repair engines:* aircraft mechanics, diagnose defective engines, disassemble engines, read standard blueprints, repair engines, technical drawings, use power tools, use testing equipment
 - *ensure compliance with airport security measures:* ensure compliance with airport security measures
 - *manage health and safety standards:* manage health and safety standards
 - *electronics:* electricity, electromechanics, electronics, engine components, mechanics
 - *install electrical and electronic equipment:* 

In [29]:
input_str = "database management systems"
for cluster_id in np.unique(affprop.labels_):
    exemplar = words[affprop.cluster_centers_indices_[cluster_id]]
    cluster = np.unique(words[np.nonzero(affprop.labels_==cluster_id)])
    if input_str in words[np.nonzero(affprop.labels_==cluster_id)]:
        cluster_str = ", ".join(cluster)
        print(" - *%s:* %s" % (exemplar, cluster_str))